In [3]:
## IMPORT LIBRARIES ----------------------------------------------------------------------------------------------------

%matplotlib qt

%reload_ext autoreload
%autoreload 3

import sys                                                                                                              # Import sys to add paths to libraries
import matplotlib.pyplot as plt                                                                                         # Import matplotlib.pyplot to plot figures
import tkinter as tk                                                                                                    # Import TK to open folder dialogs to select folders
from tkinter import filedialog                                                                                          # Import filedialog to open folder dialogs to select folders
import numpy                                                                                                            # Import numpy to work with arrays and make calculations
from shutil import rmtree
from datetime import datetime, timedelta                                                                                # Import time to measure time 
import time                                                                                                             # Import time to measure time
import os                                                                                                               # Import path to work with paths
import pandas                                                                                                           # Import pandas to work with dataframes
import warnings                                                                                                         # Import warnings to ignore warnings
warnings.filterwarnings('ignore')                                                                                       # Ignore warnings

## IMPORT CIRCADIPY ----------------------------------------------------------------------------------------------------

sys.path.append('E:\\github\\nnc-ufmg\\circadipy\\src')
import chrono_reader as chr                                                                                             # Import chrono_reader to read data
import chrono_plotter as chp                                                                                            # Import chrono_plotter to plot data
import chrono_rythm as chrt                                                                                             # Import chrono_rithm to make calculations
import chrono_simulation as chs                                                                                         # Import chrono_simulation to simulate data

PyMICE library v. 1.2.1

The library is available under GPL3 license; we ask that reference to our paper
as well as to the library itself is provided in any published research making
use of PyMICE. Please run:

>>> print(pm.__REFERENCING__)

for more information (given that the library is imported as `pm`).




In [46]:
## SET THE ENVIRONMENT -------------------------------------------------------------------------------------------------

root = tk.Tk()                                                                                                          # Create the root window
root.attributes('-topmost',True)                                                                                        # Keep the window on top of others
root.iconify()                                                                                                          # Hide the root window

database_files = filedialog.askopenfilenames(title='Select the database file(s)')                                       # Ask the user to select the database file(s)

main_save_folders = [os.path.dirname(file) for file in database_files]                                                  # Get the folder where the database file is located
root.destroy()                                                                                                          # Destroy the root window

for folder in main_save_folders:                                                                                        # For each folder
    print('The database file is located in: ' + folder)                                                                 # Print the folder where the database file is located

The database file is located in: E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadb
The database file is located in: E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadb


In [47]:
def reset_folder(folder):                                                                                               # Function to reset the folder
    for root, dirs, files in os.walk(folder):                                                                           # For each file in the folder
        for f in files:                                                                                                 # For each file
            os.unlink(os.path.join(root, f))                                                                            # Remove the file
        for d in dirs:                                                                                                  # For each folder
            rmtree(os.path.join(root, d))                                                                               # Remove the folder

In [49]:
## READ THE DATABASE ---------------------------------------------------------------------------------------------------

remove_file = False                                                                                                     # Set if the file should be removed or not if it already exists

database = {}                                                                                                           # Create a dictionary with the database
save_folders = {}                                                                                                       # Create a dictionary with the save folder
result_folders = {}                                                                                                     # Create a dictionary with the results folder

for file in database_files:
    test = file.split('/')[-1].split('_')[-1]                                                                           # Get the parameter name from the file name
    test = test.split('.')[0]                                                                                           # Remove the extension from the file name
    print('Getting data from: ' + test)                                                                                 # Print the parameter name
    database[test] = {}                                                                                                 # Create a dictionary for the parameter

    if test == 'jtk':
        prefix = 'JTK'
    elif test == 'lombscargle':
        prefix = 'LombScargle_'

    main_save_folder = os.path.dirname(file)
    save_folders[test] = main_save_folder + '/data_' + test + '/'                                                       # Create a folder to save the plots
    result_folders[test] = main_save_folder + '/results_' + test + '/'                                                  # Create a folder to save the results

    # check if the folder exists
    if not os.path.exists(save_folders[test]):
        os.makedirs(save_folders[test])
    else:
        if remove_file:                                                                                                 # If the file should be removed
            reset_folder(save_folders[test])                                                                            # Reset the folder
            print("Data folder reseted")                                                                                # Print that the folder was reseted
        else:                                                                                                           # If the file shouldn't be removed
            print("Data folder already exists")                                                                         # Print that the folder already exists

    if not os.path.exists(result_folders[test]):
        os.makedirs(result_folders[test])
    else:
        if remove_file:                                                                                                 # If the file should be removed
            reset_folder(result_folders)                                                                                # Reset the folder
            print("Results folder reseted")                                                                             # Print that the folder was reseted
        else:                                                                                                           # If the file shouldn't be removed
            print("Results folder already exists")                                                                      # Print that the folder already exists    

    database_df = pandas.read_csv(file)                                                                                 # Read the database file
    display(database_df.head())                                                                                         # Display the database

    database_df = database_df[(database_df[prefix + 'period'] >= 20) & (database_df[prefix + 'period'] <= 28)]          # Only periods beteween 20 and 28 hours

    for row in database_df.iterrows():
        database[test][row[0]] = {}
        for column in database_df.columns:
            if column == 'Time':
                split_time = row[1][column].split(';')
                database[test][row[0]][column] = [int(i) for i in split_time]
            elif column == 'Values':
                split_values = row[1][column].split(';')
                database[test][row[0]][column] = [float(i) for i in split_values]
            else: 
                database[test][row[0]][column] = row[1][column]

Getting data from: jtk
Results folder already exists
Results folder already exists


,Probeset_ID,Symbol,Time,Values,JTKP,JTKQ,JTKperiod,JTKphase,Tissue
0,10556463,Arntl,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,167.8;360.7;350.3;428.6;380.1;268.8;152.3;89.5...,5.511650e-12,1.489390e-07,24.0,0.0,heart
1,10487040,Fbn1,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,445.5;621.5;666.8;710.1;773.4;854.8;810.0;689....,1.256660e-11,1.489390e-07,24.0,3.0,heart
2,10518781,Per3,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,339.6;199.2;162.6;153.4;176.1;223.4;322.7;516....,1.256660e-11,1.489390e-07,24.0,12.0,heart
3,10433219,Nat15,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,1292.6;1167.5;1082.6;1026.7;998.5;886.5;999.9;...,2.770380e-11,2.462590e-07,26.0,17.0,heart
4,10363773,Rhobtb1,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,1101.3;1048.6;841.1;576.2;667.9;564.3;1014.2;1...,1.229230e-10,8.741310e-07,24.0,13.0,heart


Getting data from: lombscargle
Results folder already exists
Results folder already exists


,Probeset_ID,Symbol,Time,Values,LombScargle_P,LombScargle_Q,LombScargle_period,LombScargle_phase,Tissue
0,10522467,Rasl11b,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,116.5;117.3;146.4;215.8;205.6;223.5;191.9;214....,0.000837,1.0,23.892,3.75,scn_2014
1,10553092,Dbp,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,197.2;239.5;231.3;249.7;280.9;311.7;318.2;341....,0.000988,1.0,23.892,7.03,scn_2014
2,10356601,Per2,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,144.4;145.4;149.5;143.4;165.9;197.2;215.8;271....,0.001150,1.0,23.892,9.81,scn_2014
3,10350733,Rgs16,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,434.7;857.4;881.5;1187.6;1041.1;1336.1;1221.0;...,0.001300,1.0,23.892,3.70,scn_2014
4,10449741,Sik1,18;20;22;24;26;28;30;32;34;36;38;40;42;44;46;4...,184.0;201.4;214.4;278.9;294.9;269.4;273.2;320....,0.001480,1.0,23.064,3.77,scn_2014


In [50]:
## CREATE A GENERIC ASC TO CIRCADIPY -----------------------------------------------------------------------------------

# Set a generic date with the format MM/DD/YY,HH:MM:SS
generic_date = '01/01/20,00:00:00'
generic_date = datetime.strptime(generic_date, '%m/%d/%y,%H:%M:%S')

# Set the generic time with the format HH:MM:SS with the times (in hours) in the database
for test in database:
    for id in database[test]:     
        database[test][id]['circadipy'] = [generic_date + timedelta(hours=t) for t in database[test][id]['Time']]
        database[test][id]['circadipy'] = [datetime.strftime(t, '%m/%d/%y,%H:%M:%S') for t in database[test][id]['circadipy']]
        database[test][id]['circadipy'] = [t + ',' + str(v) for t, v in zip(database[test][id]['circadipy'], database[test][id]['Values'])]
        file_name = str(database[test][id]['Probeset_ID']) + '_' + str(database[test][id]['Symbol']) + '.asc'
        file_to_save = os.path.join(save_folders[test], file_name)
        database[test][id]['file_name'] = file_to_save
        with open(file_to_save, 'w') as f:
            f.write('\n'.join(database[test][id]['circadipy']))
        f.close()

In [51]:
## CONFIGURE THE EXPERIMENT --------------------------------------------------------------------------------------------

zt_0_time = 0                                                                                                           # Set the time for the first ZT (0)
type = 'generic'                                                                                                        # Set the type of the data decoding (endocrino or intellicage)
labels_dict = {'cycle_types': ['LD'], 'test_labels': ['Gene'], 'cycle_days': []}                                        # Create a dictionary to store to pass as argument to the read_protocol function

## READ THE PROTOCOL ---------------------------------------------------------------------------------------------------

for test in database:
    for id in database[test]:     
        file = database[test][id]['file_name']
        name = str(database[test][id]['Probeset_ID']) + '_' + str(database[test][id]['Symbol'])
        database[test][id]['protocol'] = chr.read_protocol(name, file, zt_0_time = zt_0_time, labels_dict = labels_dict, 
                                                            type = type, consider_first_day = True)                     # Read the protocol (chrono_reader.py)
        database[test][id]['protocol'].normalize_data(type = 'minmax', per_day = False)                                 # Normalize the data (chrono_reader.py)
        database[test][id]['protocol'].apply_filter(window = 3, type = 'moving_average', order = 2, reverse = False)    # Apply a filter to the data (chrono_reader.py)

In [52]:
## SET THE PARAMETERS FOR THE COSINOR FITTING --------------------------------------------------------------------------

dict = {'time_shape': 'continuous',                     
        'step': 0.01, 
        'start_time': 20, 
        'end_time': 28, 
        'n_components': [1]}                                                                                            # Create a dictionary to pass as argument to the fit_cosinor function

## FIT THE COSINOR TO THE DATA -----------------------------------------------------------------------------------------

init = time.time()                                                                                                      # Get the initial time

for test in database:
    for id in database[test]:     
        save_folder = result_folders[test]                                                                              # Create a folder to save the results
        protocol = database[test][id]['protocol']
        best_models, _ = chrt.fit_cosinor(protocol, dict = dict, save_folder = save_folder)                             # Fit the cosinor to the data (chrono_rithm.py)
        database[test][id]['best_models'] = best_models
        chp.model_over_signal(protocol, best_models, position = 'head', mv_avg_window = 1, save_folder = save_folder,
                              format = 'svg')                                                                           # Plot representative figure with the data and the respective model (using chrono_plotter.py)

end = time.time() - init                                                                                                # Get the time elapsed

In [53]:
## COMPARE THE COSINOR FITTING WITH THE GORUND-TRUTH PERIODS -----------------------------------------------------------

for test in database:
    period_error = []                                                                                                   # Create an empty list to store the model error
    phase_error = []                                                                                                    # Create an empty list to store the model error
    p_error = []                                                                                                        # Create an empty list to store the model error
    comparison_file = result_folders[test] + 'comparison_' + test + '.txt'

    if 'comparison_' + test + '.txt' in os.listdir(result_folders[test]):
        os.remove(comparison_file)

    for id in database[test]:     
        if id != 'save_folder':
            if test == 'jtk':
                prefix = 'JTK'
                plot_name = 'JTK'
            elif test == 'lombscargle':
                prefix = 'LombScargle_'
                plot_name = 'Lomb-Scargle'
            
            name = database[test][id]['Symbol']

            period_circadipy = database[test][id]['best_models']['period'][0]                                           # Get the activity period calculated by the model
            period_other = database[test][id][prefix + 'period']                                                        # Get the activity period used to generate the data
            
            phase_circadipy = database[test][id]['best_models']['acrophase_zt'][0]                                      # Get the activity phase calculated by the model
            phase_other = database[test][id][prefix + 'phase']                                                          # Get the activity phase used to generate the data
            
            phase_circadipy = phase_circadipy - 6
            if phase_circadipy < 0:
                phase_circadipy += 24
            if phase_other < 0:
                phase_other += 24
            if phase_circadipy < 12 and phase_other > 12:
                phase_circadipy += 24            

            period_error.append(period_circadipy - period_other)                                                        # Calculate the model error for the period
            phase_error.append(phase_circadipy - phase_other)                                                           # Calculate the model error for the phase

            if phase_error[-1] > 12:
                phase_circadipy -= period_circadipy
                phase_error[-1] = phase_circadipy - phase_other
            elif phase_error[-1] < -12:
                phase_other -= period_other
                phase_error[-1] = phase_circadipy - phase_other

            with open(comparison_file, 'a') as f:
                f.write('\n------------------------------------------------------------------')
                f.write('\nGene: ' + str(name) + ' (' + test + ')')                                                     # Print the gene name
                f.write('\nCircadiPy: ' + str(period_circadipy) + ' Other: ' + str(period_other))                       # Print the period calculated by the model and the ground-truth period
                f.write('\nThe period error for ' + str(id) + ' is: ' + str(period_error[-1]))                          # Print the model error for the period
                f.write('\nCircadiPy: ' + str(phase_circadipy) + ' Other: ' + str(phase_other))                         # Print the phase calculated by the model and the ground-truth phase
                f.write('\nThe phase error for ' + str(id) + ' is: ' + str(phase_error[-1]))                            # Print the model error for the phase
            f.close()

    period_error = numpy.array(period_error)                                                                            # Convert the list to a numpy array
    phase_error = numpy.array(phase_error)                                                                              # Convert the list to a numpy array
    
    mean_period_error = numpy.mean(period_error)                                                                        # Calculate the mean error for the period
    mean_phase_error = numpy.mean(phase_error)                                                                          # Calculate the mean error for the phase

    std_period_error = numpy.std(period_error)                                                                          # Calculate the standard deviation for the period
    std_phase_error = numpy.std(phase_error)                                                                            # Calculate the standard deviation for the phase

    with open(comparison_file, 'a') as f:
        f.write('\n==================================================================')
        f.write('\nThe mean error for the period parametrization is: ' + str(round(mean_period_error, 4)) \
                + ' +/- ' + str(round(std_period_error, 4)))                                                            # Print the mean error for the period
        f.write('\nThe mean error for the phase parametrization is: ' + str(round(mean_phase_error, 4)) \
                + ' +/- ' + str(round(std_phase_error, 4)))                                                             # Print the mean error for the phase
    f.close()

    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('Parametrization Error (CircadiPy - ' + plot_name + ')', fontsize=16)
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.hist(period_error, bins=16, color='midnightblue', edgecolor='midnightblue')
    ax1.set_title('PERIOD\nMEAN: ' + str(round(mean_period_error,2)) + ' +/- ' + str(round(std_period_error,2)), fontsize=16)
    ax1.set_xlabel('PERIOD (HOUR)', fontsize=16)
    ax1.set_ylabel('FREQUENCY', fontsize=16)
    ax1.set_xlim(-3.5, 3.5)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.hist(phase_error, bins=16, color='midnightblue', edgecolor='midnightblue')
    ax2.set_title('PHASE\nMEAN: ' + str(round(mean_phase_error,2)) + ' +/- ' + str(round(std_phase_error,2)), fontsize=16)
    ax2.set_xlabel('PHASE (HOUR)', fontsize=16)
    ax2.set_xlim(-3.5, 3.5)
    plt.tight_layout()
    
    plt.savefig(result_folders[test] + 'parametrization_error_' + test + '.svg', dpi=300)
    plt.close()